In [1]:
import tensorflow as tf
import numpy as np

**Build stage

In [2]:
n_inputs=28*28
n_hidden1=300
n_hidden2=100
n_outputs=10
#None because we don't know train set size
X=tf.placeholder(tf.float32,shape=(None,n_inputs),name="X")
y=tf.placeholder(tf.int64,shape=(None),name="y")

In [3]:
#neuron layer
def neuron_layer(X,n_neurons,name,activation=None):
    #tidy structure for tensorboard
    with tf.name_scope(name):
        
        #inputs count
        n_inputs=int(X.get_shape()[1]) #784=28*28
        
        #weight matrix (layer kernel)
        stddev=2/np.sqrt(n_inputs+n_neurons) #truncated normal distribution
        init=tf.truncated_normal((n_inputs,n_neurons),stddev=stddev) #random values
        W=tf.Variable(init,name="kernel")
        
        #bias
        b=tf.Variable(tf.zeros([n_neurons]),name="bias")
        
        #weighted sums of inputs + bias
        Z=tf.matmul(X,W)+b
        
        if activation is None:
            return Z
        else:
            return activation(Z)        

In [4]:
#deep neural network
with tf.name_scope("dnn"):
    
    #X ==> hidden1
    hidden1=neuron_layer(X,n_hidden1,name="hidden1",activation=tf.nn.relu)
    
    #hidden1 ==> hidden2
    hidden2=neuron_layer(hidden1,n_hidden2,name="hidden2",activation=tf.nn.relu)
    
    #hidden2 ==> outputs
    logits=neuron_layer(hidden2,n_outputs,name="outputs")
    
    #neuron_layer() can be raplaced to tf.layers.dense()

In [5]:
#calculate loss for training (loss function)
with tf.name_scope("loss"):
    xentropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)
    loss=tf.reduce_mean(xentropy,name="loss")

In [6]:
learning_rate=0.01
#optimizer for loss function
with tf.name_scope("train"):
    optimizer=tf.train.GradientDescentOptimizer(learning_rate)
    training_op=optimizer.minimize(loss)

In [7]:
#does our network answer rithg?
with tf.name_scope("eval"):
    
    #boolean tensor
    correct=tf.nn.in_top_k(logits,y,1)
    
    #middle value
    accuracy=tf.reduce_mean(tf.cast(correct,tf.float32))

In [8]:
#clear namespace for tensorboard
init=tf.global_variables_initializer()
#save model to disk
saver=tf.train.Saver()

**Execution stage

In [9]:
#load data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [10]:
#set network parameters
n_epochs=40
batch_size=50

In [11]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [ ]:
#training
#open session
with tf.Session() as sess:
    #init all variables
    init.run()
    #step each epoch
    for epoch in range(n_epochs):
        #extract each mini-batch corresponding training set
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            #perform training operation, using input data mini-batch and goals
            sess.run(training_op,feed_dict={X:X_batch,y:y_batch})
        #last mini-batch model evaluation
        acc_batch=accuracy.eval(feed_dict={X:X_batch,y:y_batch})
        #full training set model evaluation
        acc_val=accuracy.eval(feed_dict={X:X_valid,y:y_valid})
        print(epoch,"training accuracy: ",acc_batch,"test accuracy: ",acc_val)
        #save parameters
        save_path=saver.save(sess,"./my_model_final.ckpt")

**Using neural network

imagePath="data/test/1/myFourBlack28.png"
imageRaw = tf.read_file(imagePath)
print(imageRaw)
print(repr(imageRaw)[:100]+"...")

imageTensor = tf.image.decode_image(imageRaw)
print(imageTensor.shape)
print(imageTensor.dtype)
#print(X_new_scaled)

image = tf.image.decode_png(imageRaw, channels=3)
print(image)

predictionDataSet=tf.data.Dataset.from_tensor_slices(["data/test"])
predictionDataSet

In [51]:
def printSymbol(symbolData):
    for y_axis in range(28):
        stringCurrent=""
        for x_axis in range(28):
            valueCurrent=symbolData[0][y_axis*28+x_axis]
            if valueCurrent==0:
                stringCurrent+="-"
            else:
                stringCurrent+="#"
        print(stringCurrent)

In [61]:
import numpy as np
from PIL import Image

#imagePath="data/test/1/myFourBlack28.png"
imagePath="data/test/1/newsymbol28.png"
img = Image.open(imagePath)
X_new_scaled = np.asarray(img, dtype=np.float32)
X_new_scaled=X_new_scaled[:,:,0]
X_new_scaled=np.reshape(X_new_scaled,[1,28*28])
X_new_scaled/=255
printSymbol(X_new_scaled)

----------------------------
-------------###------------
------------#####-----------
------------#####-----------
-----------######-----------
-----------######-----------
-----------######-----------
-----------######-----------
------------#####-----------
-----------######-----------
-----------#######----------
-----------#######----------
-----------#######----------
--------------#####---------
--------------#####---------
--------------#####---------
-------##-----#####---------
-------###----#####---------
-------###----#####---------
-------####---#####---------
-------####--#####----------
-------####-######----------
-------###########----------
-------##########-----------
-------#########------------
-------########-------------
--------######--------------
----------------------------


In [63]:
#open session
with tf.Session() as sess:
    #load model
    saver.restore(sess,"./my_model_final.ckpt")
    #a series of new images for classification, scaled within 0 and 1
    #X_new_scaled=[...]
    #evaluate logits node
    Z=logits.eval(feed_dict={X:X_new_scaled})
    #call argmax function to made a class prognose. Or call softmax to know all class evaluation probability
    y_pred=np.argmax(Z,axis=1)

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


In [64]:
print(y_pred)

[3]
